In [1]:
import pandas as pd
import pickle as pkl
import statistics
#from basketball_reference_web_scraper import client
#from basketball_reference_web_scraper.data import OutputType
#import hockey_scraper
import datetime
import requests
from sportsipy.nfl.boxscore import Boxscore, BoxscorePlayer
import warnings
from bs4 import BeautifulSoup, Comment
import time
import statistics
#from sklearn.metrics import mean_squared_error
#import matplotlib.pyplot as plt
import re, os
import time
import random
import datetime as dt
from datetime import datetime, timedelta
from IPython.display import clear_output
from baseball_scraper import batting_stats_range, pitching_stats_range, baseball_reference
from sportsipy.mlb.boxscore import Boxscores
warnings.simplefilter("ignore")

THINGS THAT STILL NEED TO BE FIXED BUT DO NOT CURRENTLY IMPACT PERFORMANCE:

    1. BE ABLE TO CALCULATE MLB SITTING POINTS
    2. BE ABLE TO ACCOUNT FOR SHOHEI OTHANI -- WE NEED TO PULL SPECIFIC STATS FOR A PLAYER DEPENDING ON WHETHER THEY ARE ROSTERED AS A PITCHER OR A HITTER

# General Imports and Functions

In [2]:
# import important files
nhl_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="NHL Name Conversions", header = 2).iloc[:, 3:]

nfl_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="NFL Name Conversions", header = 2).iloc[:, 1:]

mlb_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="MLB Name Conversions", header = 2).iloc[:, 3:]
nba_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="NBA Name Conversions", header = 2).iloc[:, 3:]

In [3]:
def findTables(url):
    """Pulls all the relevant table ids from sports reference"""
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.findAll('div', id = "content")
    divs = divs[0].findAll("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.findAll("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)

In [4]:
def find_nhl_season(day, month, year):
    """The url for scraping NHL games channges depending on which season were in, but the season doesn't directly correlate to the year. This returns the correct year to use in the URL based on the date
       If we play another season in 2023, make sure to update with further dates"""
    
    
    date = str(year) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    if date > "2022-06-30":
        return "2023"
    elif date > "2021-07-07":
        return "2022"
    else:
        return "2021"

# Gathering Daily Stats

## NBA Statistics Collection

In [5]:
def get_daily_nba_stats(day, month, year):
    """A function that will return a cleaned pandas dataframe of daily NBA statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       Returns: A dictionary with three keys of 'Players', 'Goalies', and 'Teams'"""
                
    
    # We will eventually return this df
    final_stats = pd.DataFrame()
    final_team_df = pd.DataFrame()

    # We will continue taking dates in intigers for consistiency. However, when scraping directly from Hockey Reference, dates will be supplied back as strings. 
    # For slicing purposes, we build our date into a correctly formatted string.
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    url_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    # Now using pandas read_html, scrape  the title info for all nhl games during the season, and slice just the ones for the given date
    season = str(int(find_nhl_season(day, month, year))) #WRITE A NEW FUNCTION THAT USES FINAL NBA SEASON DATES
    all_nbl_games_url = "https://www.basketball-reference.com/leagues/NBA_{}_games.html".format(season)
    webpage = requests.get(all_nbl_games_url)
    soup = BeautifulSoup(webpage.content, "html.parser")
    months = [x.text.lower() for x in soup.find_all("div", {"class":"filter"})[0].find_all('a')]
    all_nba_games = pd.DataFrame()
    for month in months:
        month_url = "https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html".format(season, month)
        monthly_games_df = pd.read_html(month_url)[0]
        all_nba_games = all_nba_games.append(monthly_games_df)
       
    all_nba_games.Date = all_nba_games.Date.apply(lambda x: datetime.strptime(x, "%a, %b %d, %Y").strftime("%Y-%m-%d"))
    relevant_nba_games = all_nba_games[all_nba_games.Date == given_date]
  
    
    # If there were no games played on the given day, the df will be empty, and throw an error if we don't break before the next section
    if len(relevant_nba_games) <= 0:
        return pd.DataFrame()
    else:
        # Next, we build the url for the specific hockey reference url using data from each game in relevant_nhl_games
        for game in range(len(relevant_nba_games)):
            
            eventual_away_table_num = 9 if relevant_nba_games["Unnamed: 7"].iloc[game] == "OT" else 8
            
            home_team = relevant_nba_games["Home/Neutral"].iloc[game]
            home_abbr = nba_name_conversions[nba_name_conversions["Team Name"] == home_team]["Team Abbreviation"].iloc[0]
            away_team = relevant_nba_games["Visitor/Neutral"].iloc[game]
            away_abbr = nba_name_conversions[nba_name_conversions["Team Name"] == away_team]["Team Abbreviation"].iloc[0]
            
            url_insert = "".join(url_date.split("-")) + "0" + home_abbr
            game_url = "https://www.basketball-reference.com/boxscores/" + url_insert + ".html"
            
            game_tables = pd.read_html(game_url, header=1)
            
            away_stats = game_tables[0]
            away_stats = away_stats[(away_stats.Starters != "Reserves") & (pd.isna(away_stats.MP) == False)]
            away_totals = away_stats[away_stats.Starters == "Team Totals"]
            away_stats = away_stats[away_stats.Starters != "Team Totals"]
            away_stats["team"] = away_abbr
            
            home_stats = game_tables[eventual_away_table_num]
            home_stats = home_stats[(home_stats.Starters != "Reserves") & (pd.isna(home_stats.MP) == False)]
            home_totals = home_stats[home_stats.Starters == "Team Totals"]
            home_stats = home_stats[home_stats.Starters != "Team Totals"]
            home_stats["team"] = home_abbr
            
            
            home_totals["team"] = home_abbr
            away_totals["opponent"] = home_abbr
            
            home_totals["opponent"] = away_abbr
            away_totals["team"] = away_abbr
            
            home_points = home_totals.PTS.iloc[0]
            away_points = away_totals.PTS.iloc[0]
        
            home_totals["PTS"] = home_points
            away_totals["PTS"] = away_points
            
            home_totals["is_win"] = 1 if home_points > away_points else 0
            away_totals["is_win"] = 1 if away_points > home_points else 0

            webpage = requests.get(game_url)
            soup = BeautifulSoup(webpage.content, "html.parser")

            
            home_ids = [z["data-append-csv"] for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][eventual_away_table_num].find_all("th") if "csk" in str(y)]]
            away_ids = [z["data-append-csv"] for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][0].find_all("th") if "csk" in str(y)]]
           
            home_stats["id"] = home_ids
            home_stats = home_stats[(home_stats.MP != "Did Not Dress") & (home_stats.MP != "Not With Team") & (home_stats.MP != "Did Not Play") & (pd.isna(home_stats.MP) == False)]
            
            away_stats["id"] = away_ids
            away_stats = away_stats[(away_stats.MP != "Did Not Dress") & (away_stats.MP != "Not With Team") & (away_stats.MP != "Did Not Play") & (pd.isna(home_stats.MP) == False)]
            
            stats_addition = home_stats.append(away_stats)
            final_stats = final_stats.append(stats_addition).reset_index(drop=True)
            
            team_totals = home_totals.append(away_totals)
            final_team_df = final_team_df.append(team_totals).reset_index(drop = True)
            
    final_stats.MP = final_stats.MP.apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    #final_stats.PTS = final_stats.MP.apply(lambda x: int(x))
    
    final_team_df.PTS = final_team_df.PTS.apply(lambda x: int(x))
    #final_team_df.MP = final_team_df.MP.apply(lambda x: float(x["players"]["MP"].iloc[0].split(":")[0]) + float(x["players"]["MP"].iloc[0].split(":")[1])/60)
    
    return {"players":final_stats, "teams":final_team_df, "date":given_date}

## NHL Stats Collection

In [6]:
def get_daily_nhl_stats(day, month, year):
    """A function that will return a cleaned pandas dataframe of daily NHL statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       Returns: A dictionary with three keys of 'Players', 'Goalies', and 'Teams'"""
    
    def pull_hits_and_blocks(player, stat, location):
        if location == "home":
            try:
                ans = home_advanced_stats[home_advanced_stats.Player == player][stat].iloc[0]
            except:
                return 0
        else:
            try: 
                ans = away_advanced_stats[away_advanced_stats.Player == player][stat].iloc[0]
            except:
                return 0
            
        return ans

                
    
    # We will eventually return this df
    final_skater_df = pd.DataFrame()
    final_goalie_df = pd.DataFrame()
    final_team_df = pd.DataFrame()
    final_penalty_df = pd.DataFrame()

    # We will continue taking dates in intigers for consistiency. However, when scraping directly from Hockey Reference, dates will be supplied back as strings. 
    # For slicing purposes, we build our date into a correctly formatted string.
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    # Now using pandas read_html, scrape  the title info for all nhl games during the season, and slice just the ones for the given date
    season = find_nhl_season(day, month, year)
    all_nhl_games_url = "https://www.hockey-reference.com/leagues/NHL_{}_games.html".format(season)
    all_nhl_games = pd.read_html(all_nhl_games_url)[0]
    try:
        playoff_games = pd.read_html(all_nhl_games_url)[1]
        all_nhl_games = all_nhl_games.append(playoff_games)
    except:
        None
    
    all_nhl_games = all_nhl_games.rename(columns = {"Date":"date", "Visitor":"visitor", "G":"away_goals",
                                                                            "Home":"home", "G.1":"home_goals", "Unnamed: 5":"is_ot",
                                                                         "Att.":"attendance", "LOG":"game_length", "Notes":"notes"})
    
    relevant_nhl_games = all_nhl_games[all_nhl_games.date == given_date]

    # If there were no games played on the given day, the df will be empty, and throw an error if we don't break before the next section
    if len(relevant_nhl_games) <= 0:
        return pd.DataFrame()
    
    else:
        
        # Next, we build the url for the specific hockey reference url using data from each game in relevant_nhl_games
        for game in range(len(relevant_nhl_games)):
            
            home_team = relevant_nhl_games.home.iloc[game]
            home_abbr = nhl_name_conversions[nhl_name_conversions["Team Name"] == home_team]["Team Abbreviation"].iloc[0]
            away_team = relevant_nhl_games.visitor.iloc[game]
            away_abbr = nhl_name_conversions[nhl_name_conversions["Team Name"] == away_team]["Team Abbreviation"].iloc[0]
            
            url_insert = "".join(given_date.split("-")) + "0" + home_abbr
            game_url = "https://www.hockey-reference.com/boxscores/" + url_insert + ".html"
            
            
            # Using the built url, scrape Hockey Reference and grab the tables (we know the order of them) for home and away
            # Stats for both skaters and goalies
            game_tables = pd.read_html(game_url, header=1)
        
            home_skater_stats = game_tables[4].drop(columns = "Rk")
            home_skater_stats["team"] = home_abbr
            
            home_goalie_stats = game_tables[5].drop(columns = "Rk")
            home_goalie_stats["team"] = home_abbr
            
            away_skater_stats = game_tables[2].drop(columns = "Rk")
            away_skater_stats["team"] = away_abbr
            
            away_goalie_stats = game_tables[3].drop(columns = "Rk")
            away_goalie_stats["team"] = away_abbr
            
            # Get and insert Pro Baseball Reference IDs into the stats dfs
            webpage = requests.get(game_url)
            soup = BeautifulSoup(webpage.content, "html.parser")
            
            
            away_names = [j.split(",")[1] + " " + j.split(",")[0] for j in [str(z).split("csk")[2].split('" ')[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][0].find_all("tr") if "csk" in str(y)]]]
            away_ids = [str(z).split("data-append-csv")[1].split(" ")[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][0].find_all("tr") if "csk" in str(y)]]
            home_names = [j.split(",")[1] + " " + j.split(",")[0] for j in [str(z).split("csk")[2].split('" ')[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][2].find_all("tr") if "csk" in str(y)]]]
            home_ids = [str(z).split("data-append-csv")[1].split(" ")[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][2].find_all("tr") if "csk" in str(y)]]
            
            away_id_dict = {away_names[n]:away_ids[n] for n in range(len(away_names))}
            home_id_dict = {home_names[n]:home_ids[n] for n in range(len(home_names))}
            
            home_skater_stats["player_id"] = home_skater_stats.Player.apply(lambda x: home_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            home_goalie_stats["player_id"] = home_goalie_stats.Player.apply(lambda x: home_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            
            away_skater_stats["player_id"] = away_skater_stats.Player.apply(lambda x: away_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            away_goalie_stats["player_id"] = away_goalie_stats.Player.apply(lambda x: away_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            
            
            
            # Add home and away goals scored to later determine which team won
            home_skater_stats["team_goals_scored"] = home_skater_stats[home_skater_stats.Player == "TOTAL"].G.sum()
            away_skater_stats["team_goals_scored"] = away_skater_stats[away_skater_stats.Player == "TOTAL"].G.sum()
            
            home_skater_stats["team_goals_allowed"] = away_skater_stats[away_skater_stats.Player == "TOTAL"].G.sum()
            away_skater_stats["team_goals_allowed"] = home_skater_stats[home_skater_stats.Player == "TOTAL"].G.sum()
            
            
            # Grab the advanced stats table to later get hits and blocks
            advanced_game_tables = pd.read_html(game_url)
            
            home_advanced_stats = advanced_game_tables[13]
            home_advanced_stats["team"] = home_abbr
            
            
            away_advanced_stats = advanced_game_tables[6]
            away_advanced_stats["team"] = away_abbr
            
            # Pull hits and blocks data from a seperate table grabbed above
            home_skater_stats["hits"] = home_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "HIT", "home"), axis = 1)
            home_skater_stats["blocks"] = home_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "BLK", "home"), axis = 1)
            
            
            away_skater_stats["hits"] = away_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "HIT", "away"), axis = 1)
            away_skater_stats["blocks"] = away_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "BLK", "away"), axis = 1)
            
            
            
            # Now that we have the stats from the game for both home and away skaters and goalies, we can add them game by game (in the loop)
            # to our final tables and rename them as needed
            skater_addition = pd.concat([home_skater_stats, away_skater_stats])
            final_skater_df = final_skater_df.append(skater_addition).reset_index(drop=True)
            
            goalie_addition = pd.concat([home_goalie_stats, away_goalie_stats])
            final_goalie_df = final_goalie_df.append(goalie_addition)
            
            # Now grab penalty data
            penalty_table = pd.read_html(game_url)[1]
            penalty_table.columns = ["time", "team", "player","penalty","penalty_minutes"]
            
            
#             .rename(columns = {"1st Period": "time", "1st Period.1": "team", "1st Period.2":"player",
#                                                                         "1st Period.3": "penalty", "1st Period.4":"penalty_minutes"})
            
            penalty_table = penalty_table[penalty_table.player.str.contains("Period") == False].drop(columns=["time"])
            final_penalty_df = final_penalty_df.append(penalty_table)
            
    
            
               
    
    # Finally, after adding together all the night's games' stats, we can make final visual edits to the tables, and add columns for some extra stats
    final_skater_df = final_skater_df.rename(columns = {"Player":"player", "G":"goals", "A": "assists", "PTS":"points", "+/-":"plus_minus",
                                                        "PIM": "penalty_minutes", "EV":"even_strength_goals", "PP":"power_play_goals",
                                                        "SH":"short_handed_goals", "GW":"game_winning_goals", "EV.1":"even_strength_assists",
                                                         "PP.1":"power_play_assists", "SH.1": "short_handed_assists", "S":"shots", "TOI":"time_on_ice"})
    final_skater_df["power_play_points"] = final_skater_df.power_play_goals + final_skater_df.power_play_assists
    final_skater_df["short_handed_points"] = final_skater_df.short_handed_goals + final_skater_df.short_handed_assists
    final_skater_df["time_on_ice"] = final_skater_df["time_on_ice"].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60 if pd.isna(x) == False else 0)
    final_skater_df = final_skater_df[['player', 'goals', 'assists', 'points', "power_play_points", "short_handed_points"] + [x for x in final_skater_df if x not in ['player', 'goals', 'assists', 'points', "power_play_points", "short_handed_points"]]]
    
    final_goalie_df = final_goalie_df.rename(columns = {"Player":"player", "DEC":"decision", "GA":"goals_against", "SA":"shots_against",
                                                        "SV":"saves", "SV%":"save_percentage", "SO":"shutouts", "PIM":"penalty_minutes",
                                                        "TOI":"time_on_ice"})
    final_goalie_df["time_on_ice"] = final_goalie_df["time_on_ice"].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60 if pd.isna(x) == False else 0)
    
    # As one last check, the scraping from Hockey Reference relies on the tables being in the right order. If they ever switch, the rename wont catch
    # this, it will fail, and we know to grab a different numbered table from the website for home, away skater stats
    correct_table_check = final_skater_df["power_play_goals"]
    
    
    # Now that we have skater and goalie stats for the home and away team finalized, grab team stats from rows titled "TOTAL"
    team_stats = final_skater_df[final_skater_df.player == "TOTAL"].dropna(axis=1).reset_index(drop=True).drop(columns = ["player", "time_on_ice"])
    team_stats = team_stats[["team"] + [x for x in team_stats if x not in ["team"]]]
    
    final_skater_df = final_skater_df[final_skater_df.player != "TOTAL"]
    final_goalie_df = final_goalie_df[final_goalie_df.player != "TOTAL"]
    
    
    
    return {"Skaters":final_skater_df, "Goalies":final_goalie_df, "Teams":team_stats, "Penalties":final_penalty_df, "date":given_date}

## NFL Stats Collection

In [7]:
def get_daily_nfl_stats(day, month, year):
    """A function that will return a cleaned pandas dataframe of daily NFL statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       """ 
    
    # We will eventually return this df
    final_offensive_df = pd.DataFrame()
    final_kicking_df = pd.DataFrame()
    final_defensive_df = pd.DataFrame()
    final_team_df = pd.DataFrame()
    final_snap_counts_df = pd.DataFrame()
    final_field_goal_df = pd.DataFrame()
    final_returns_df = pd.DataFrame()
    final_penalty_df = pd.DataFrame()
    

    # We will continue taking dates in intigers for consistiency. However, when scraping directly from Football Reference, dates will be supplied back as strings. 
    # For slicing purposes, we build our date into a correctly formatted string.
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    # Now using pandas read_html, scrape  the title info for all nfl games during the season, and slice just the ones for the given date
    all_nfl_games_url = "https://www.pro-football-reference.com/years/2022/games.htm"
    all_nfl_games = pd.read_html(all_nfl_games_url)[0].rename(columns = {"Date":"date", "Week":"week", "Day":"day", "Time":"time", "Winner/tie":"winner/tie", "Loser/tie":"loser/tie", "PtsW":"winning_team_points",
                                                                     "PtsL":"losing_team_points", "YdsW":"winning_team_yards", "TOW":"winning_team_turnovers", "YdsL":"losing_team_yards", "TOL":"losing_team_turnovers"})
    relevant_nfl_games = all_nfl_games[all_nfl_games.date == given_date]
    
    # If there were no games played on the given day, the df will be empty, and throw an error if we don't break before the next section
    if len(relevant_nfl_games) <= 0:
        return pd.DataFrame()
    
    else:
        
        # Next, we build the url for the specific hockey reference url using data from each game in relevant_nhl_games
        for game in range(len(relevant_nfl_games)):
            
            winning_team = relevant_nfl_games["winner/tie"].iloc[game]
            winning_abbr = nfl_name_conversions[nfl_name_conversions["Team Name"] == winning_team]["Team Abbreviation"].iloc[0]
            losing_team = relevant_nfl_games["loser/tie"].iloc[game]
            losing_abbr = nfl_name_conversions[nfl_name_conversions["Team Name"] == losing_team]["Team Abbreviation"].iloc[0]
            
            # Account for the strange labeling of urls when there is a neutral site game like the SB or international
            if relevant_nfl_games.iloc[game]["Unnamed: 5"] != "N":
                url_key = winning_abbr if relevant_nfl_games.iloc[game]["Unnamed: 5"] != "@" else losing_abbr
                url_insert = "".join(given_date.split("-")) + "0" + url_key.lower()
                game_url = "https://www.pro-football-reference.com/boxscores/" + url_insert + ".htm"
            
            else:
                webpage = requests.get(all_nfl_games_url)
                soup = BeautifulSoup(webpage.content, "html.parser")
                date_insert = "".join(given_date.split("-")) + "0"
                game_possibilities = soup.find_all("td", {"data-stat":"boxscore_word"})
                games_on_date = [x for x in game_possibilities if date_insert in str(x)]
                games_with_right_team = [x for x in games_on_date if winning_abbr.lower() in str(x) or losing_abbr.lower() in str(x)]
                addition = games_with_right_team[0].a["href"]
                game_url = "https://www.pro-football-reference.com" + addition
 
            
            # Using the built url, scrape Football Reference and grab the tables (we know the order of them) for offensive, defensive, special teams and snap counts
            game_tables = pd.read_html(game_url, header=1)
        
            
            offensive_game_stats = game_tables[2].rename(columns = {"Player":"player", "Tm":"team", "Cmp":"completions", "Att":"passing_attempts", "Yds":"passing_yards", "TD":"passing_touchdowns", 
                                 "Int":"interceptions", "Sk":"sacks_taken", "Yds.1":"sack_yards_taken", "Lng":"longest_pass", "Rate":"quaterback_rating", "Att.1":"rushing_attempts",
                                 "Yds.2":"rushing_yards", "TD.1":"rushing_touchdowns", "Lng.1":"longest_rush", "Tgt":"targets", "Rec":"receptions", "Yds.3":"receiving_yards",
                                 "TD.2":"receiving_touchdowns", "Lng.2":"longest_reception", "Fmb":"total_fumbles", "FL":"fumbles_lost"}).dropna(subset = ["player"])
            offensive_game_stats = offensive_game_stats[offensive_game_stats.player != "Player"]
            
            point_tables = pd.read_html(game_url, header=0)
            point_table = point_tables[1].iloc[-1].iloc[-2:]
            offensive_game_stats["points_scored"] = offensive_game_stats.team.apply(lambda x: point_table.loc[x])
            offensive_game_stats["points_allowed"] = offensive_game_stats.team.apply(lambda x: point_table[point_table.index!=x][0])
            
            # Get and insert offensive and kicking IDs
            webpage = requests.get(game_url)
            soup = BeautifulSoup(webpage.content, "html.parser")
            
            offensive_ids = [str(x).split("data-append-csv=")[1].split(" ")[0].strip('"') for x in soup.find_all("tr") if "data-append-csv" in str(x)]
            offensive_names = [x.a.text.strip() for x in soup.find_all("tr") if "data-append-csv" in str(x)]
            offensive_id_dict = {offensive_names[n]:offensive_ids[n] for n in range(len(offensive_names))}
            offensive_game_stats["player_id"] = offensive_game_stats.player.apply(lambda z: offensive_id_dict[z])
            
            
            kicking_ids = [z.split('="')[1].split(" ")[0].strip('"') for z in [y for y in [x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "kick" in str(x)] if "data-append-csv" in str(y)][1].split("data-append-csv")[1:]]
            kicking_names = [z.split("<")[0].strip() for z in [y for y in [x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "kick" in str(x)] if "data-append-csv" in str(y)][1].split('.htm">')[1:]]
            kicking_id_dict = {kicking_names[n]:kicking_ids[n] for n in range(len(kicking_names))}
            
            defense_ids = [y.split('" ')[0].strip('"') for y in str([x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "defense" in str(x)]).split("data-append-csv=")[1:]]
            defense_names = [y.split("<")[0].replace('\\', "").strip() for y in str([x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "defense" in str(x)]).split('.htm">')[1:]]
            defense_id_dict = {defense_names[n]:defense_ids[n] for n in range(len(defense_names))}
            
            # Make a table of all scored field goals and their distances from the pd.read_html. It doesn't really go with any category, but we'll need it for kicker scoring
            # Note: we need to do a new read_html because the header level is different than the original one
            scoring = pd.read_html(game_url, header=0)[1]
            field_goals = scoring[(scoring.Detail.str.contains("field goal") == True) & (scoring.Detail.str.contains("field goal return") == False)]
            field_goals["kicker"] = field_goals.Detail.apply(lambda x: " ".join(x.split("yard")[0].split(" ")[0:-2]))
            field_goals["distance"] = field_goals.Detail.apply(lambda x: (x.split("yard")[0].split(" ")[-2]))
            field_goals["date"] = str(month) + "/" + str(day) + "/" + str(year)
            final_field_goals = field_goals[["kicker", "distance", "date"]].copy()
            final_field_goal_df = final_field_goal_df.append(final_field_goals)
            
            # Make a column for if the team won
            offensive_game_stats["won"] = offensive_game_stats.team.apply(lambda x: 1 if x==winning_abbr else 0)

            # Make alterations to offensive stats - checking for division matchup, 
            teams = offensive_game_stats.team.unique()
            divisions = [nfl_name_conversions[nfl_name_conversions["Team Code"] == n]["League"].iloc[0] + nfl_name_conversions[nfl_name_conversions["Team Code"] == n]["Division"].iloc[0] for n in teams]
            offensive_game_stats["is_division_game"] = 1 if len(set(divisions)) == 1 else 0
            final_offensive_df = final_offensive_df.append(offensive_game_stats)
            
            
            # Other than the straightforward offensive stats, the other tables (defensive, kicking, etc.) are all hidden in comments. We now take a secondary route to find them
            scraped_html = requests.get(game_url)
            soup = BeautifulSoup(scraped_html.content)

            # Get all html comments, then filter out everything that isn't a table
            comments = soup.find_all(text=lambda text:isinstance(text, Comment))
            commented_out_tables = [BeautifulSoup(cmt).find_all('table') for cmt in comments]
            
            # Some of the entries in `commented_out_tables` are empty lists. Remove them.
            commented_out_tables = [tab[0] for tab in commented_out_tables if len(tab) == 1]

            # Go through the commented out tables pulling tables for kicking, defense, and snap counts, before altering titles and formatting to make them helpful
            for table in commented_out_tables:

                if table.get('id') == 'kicking':
                    kicking_table = pd.read_html(str(table), header=1)[0].dropna(subset=["Tm"]).rename(columns = {"Player":"player", "Tm":"team", "XPM":"extra_points_made", "XPA":"extra_points_attempted",
                                                                                                                  "FGM":"field_goals_made", "FGA":"field_goals_attempted", "Pnt":"punts",
                                                                                                                  "Yds":"punt_yards", "Y/P":"yards_per_punt", "Lng":"longest_punt"}).fillna(0)
                    kicking_table = kicking_table[kicking_table.team != "Tm"]
                    kicking_table["player_id"] = kicking_table.player.apply(lambda x: kicking_id_dict[x])
                    final_kicking_df = final_kicking_df.append(kicking_table)
                    
                if table.get("id") == "player_defense":
                    defense_table = pd.read_html(str(table), header=1)[0].dropna(subset=["Tm"]).rename(columns = {"Tm":"team", "Int":"interceptions", "Yds":"interception_return_yards",
                                                                                                      "TD":"interception_touchdowns", "Lng":"longest_interception_return",
                                                                                                      "Sk":"sacks", "Comb":"total_tackles", "TFL":"tackles_for_loss",
                                                                                                      "QBHits":"qb_hits", "FR":"fumble_recoveries", "Yds.1":"fumble_return_yards",
                                                                                                      "TD.1":"fumble_touchdowns"}).drop(columns = ["PD", "Solo", "Ast", "FF"])
                    defense_table = defense_table[defense_table.team != "Tm"]
                    defense_table["player_id"] = defense_table.Player.apply(lambda x: defense_id_dict[x])
                    final_defensive_df = final_defensive_df.append(defense_table)
                    
                    
                if table.get("id") == "returns":
                    returns_table = pd.read_html(str(table), header = 1)[0].dropna(subset=["Tm"]).rename(columns = {"Player":"player", "Tm":"team", "Yds":"kick_return_yards", "Yds.1":"punt_return_yards",
                                                                                                                    "TD":"kick_return_touchdowns", "TD.1":"punt_return_touchdowns"})
                    returns_table = returns_table[returns_table.player != "Player"][["player", "team", "kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"]]
                    final_returns_df = final_returns_df.append(returns_table)
                    
                if table.get("id") == "team_stats":
                    team_stats_table = pd.read_html(str(table), header = 0)[0].rename(columns = {"Unnamed: 0":"stat"})
                    penalty_yards = team_stats_table[team_stats_table.stat == "Penalties-Yards"].T.iloc[1:]
                    penalty_yards = penalty_yards[penalty_yards.columns[0]].apply(lambda x: float(x.split("-")[-1]))
                    final_penalty_df = pd.concat([final_penalty_df, penalty_yards])
                    
                if table.get("id") == "home_snap_counts" or table.get("id") == "vis_snap_counts":
                    location = table.get("id")

                    snaps = pd.read_html(str(table), header=1)[0].rename(columns = {"Player":"player", "Num":"offensive_snaps", "Pct":"offensive_percent", "Num.1":"defensive_snaps",
                                                                                         "Pct.1":"defensive_percent", "Num.2":"special_teams_snaps", "Pct.2":"special_teams_percent"})

                    snaps["total_offensive_snaps"] = snaps.apply(lambda x: x.offensive_snaps * (100/float(x.offensive_percent.split("%")[0])) if x.offensive_percent != "0%" else 0, axis=1)
                    snaps["total_offensive_snaps"] = round(snaps[snaps.offensive_snaps == max(snaps.offensive_snaps)].total_offensive_snaps.iloc[0])

                    snaps["total_defensive_snaps"] = snaps.apply(lambda x: x.defensive_snaps * (100/float(x.defensive_percent.split("%")[0])) if x.defensive_percent != "0%" else 0, axis=1)
                    snaps["total_defensive_snaps"] = round(snaps[snaps.defensive_snaps == max(snaps.defensive_snaps)].total_defensive_snaps.iloc[0])

                    snaps["total_special_teams_snaps"] = snaps.apply(lambda x: x.special_teams_snaps * (100/float(x.special_teams_percent.split("%")[0])) if x.special_teams_percent != "0%" else 0, axis=1)
                    snaps["total_special_teams_snaps"] = round(snaps[snaps.special_teams_snaps == max(snaps.special_teams_snaps)].total_special_teams_snaps.iloc[0])

                    snaps = snaps.drop(columns = ["Pos", "offensive_percent", "defensive_percent", "special_teams_percent"])
                    
                    final_snap_counts_df = final_snap_counts_df.append(snaps)

            final_field_goal_df["team"] = final_field_goal_df.kicker.apply(lambda x: final_kicking_df[final_kicking_df.player == x].team.iloc[0])
            
    

    return {"Offense":final_offensive_df.reset_index(drop=True), "Defense":final_defensive_df.reset_index(drop=True), "Kicking":final_kicking_df.reset_index(drop=True),
            "Snaps":final_snap_counts_df.reset_index(drop=True), "Field Goals":final_field_goal_df.reset_index(drop=True), "Returns":final_returns_df.reset_index(drop=True),
            "Penalties":final_penalty_df.rename(columns = {0:"penalty_yards"}).reset_index().rename(columns = {"index":"team"}), "date":given_date}

## MLB Stats Collection

In [79]:
def get_daily_mlb_stats(day, month, year):

    """A function that will return a cleaned pandas dataframe of daily MLB statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       """ 
    def runs_allowed(row):
        return team_batting[(team_batting.game_id == row.game_id) & (team_batting["Team Name"] != row["Team Name"])].R.iloc[0]
    
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    dh_dict = {}
    
    # We will eventually return these dfs
    final_batting_df = pd.DataFrame()
    final_pitching_df = pd.DataFrame()
    final_team_df = pd.DataFrame()

    
    # Get all games played that day and their url inserts to be used in scraping baseball reference
    games = Boxscores(datetime(year, month, day)).games["{}-{}-{}".format(month, day, year)]
    
    for game in games:
        home_team = game["home_name"]
        
        if home_team not in dh_dict:
            dh_dict[home_team] = 1
        url_insert = mlb_name_conversions[mlb_name_conversions["Team Name"] == home_team]["Fangraphs Insert"].iloc[0]
        url = "https://www.fangraphs.com/boxscore.aspx?date={}&team={}&dh=0&season={}#home_standard".format(given_date, url_insert, year)
        
        try:
            tables = pd.read_html(url)
        except:
            try: 
                url = "https://www.fangraphs.com/boxscore.aspx?date={}&team={}&dh={}&season={}#home_standard".format(given_date, url_insert, dh_dict[home_team], year)
                tables = pd.read_html(url)
                dh_dict[home_team] += 1
            except:
                tables = pd.read_html(url)
        
        # Get and insert offensive and kicking IDs
        webpage = requests.get(url)
        soup = BeautifulSoup(webpage.content, "html.parser")
        
        all_ids = [y["href"].split("playerid=")[1].split("&")[0]for y in [item for sublist in [x.find_all("a") for x in soup.find_all("table", {"class":"rgMasterTable"})[0:4]] for item in sublist]]
        all_names = [y.text for y in [item for sublist in [x.find_all("a") for x in soup.find_all("table", {"class":"rgMasterTable"})[0:4]] for item in sublist]]
        
        id_dict = {all_names[n]:all_ids[n] for n in range(len(all_names))}
                
        
        # Get the basic info of the game like who played, how much they scored, and who won
        game_info = tables[14]
        home_team = game_info[0].iloc[2]
        home_runs = int(game_info[game_info.columns[-1]].iloc[2])
        away_team = game_info[0].iloc[1]
        away_runs = int(game_info[game_info.columns[-1]].iloc[1])
        winner = home_team if home_runs > away_runs else away_team
        innings = int(game_info.columns[0]) - 1
        
        game_id = home_team + str(home_runs) + away_team + str(away_runs) + given_date
        
        # Get the raw batting and pitching tables from the game
        home_batting = tables[27]
        home_pitching = tables[28]
        
        away_batting = tables[29]
        away_pitching = tables[30]
        
        
        # Combine the home and away stats for both batting and pitching
        home_batting["Team Name"] = home_team
        away_batting["Team Name"] = away_team
        
        home_pitching["Team Name"] = home_team
        away_pitching["Team Name"] = away_team
        
        batting = home_batting.append(away_batting)
        pitching = home_pitching.append(away_pitching)
        
        # Clean the batting tables before we append them to the final batting stats
        team_batting = batting[batting.Name == "Total"]
        player_batting = batting[batting.Name != "Total"]
        
        player_batting["total_bases"] = player_batting["1B"] + player_batting["2B"] * 2 + player_batting["3B"] * 3 + player_batting["HR"] * 4
        
        player_batting["Position"] = player_batting.Name.apply(lambda x: x.split(" - ")[-1])
        player_batting.Name = player_batting.Name.apply(lambda x: x.split(" - ")[0])

        player_batting["walks"] = player_batting.BB + player_batting.IBB + player_batting.HBP
        player_batting["sacrifices"] = player_batting.SF + player_batting.SH
        player_batting = player_batting.rename(columns = {"Name":"name", "PA":"plate_appearences", "H":"hits", "R":"runs", "1B":"singles", "2B":"doubles", "3B":"triples", "HR":"home_runs", "RBI":"runs_batted_in",
                                                          "SB":"stolen_bases", "SO":"strikeouts"})
        player_batting["game_id"] = game_id
        
       
        player_batting["player_id"] = player_batting.name.apply(lambda x: id_dict[x])
        
        
        # Clean the pitching tables before we append them to the final pitching stats
        pitching.Name = pitching.Name.apply(lambda x: x.split(" (")[0])
        
                # Right now we are calculating team pitching stats because it's easy, but I'm not including it in the output bc we don't have any stats with it. 
                # However, if we want to include them one day, here is where they are.
        team_pitching = pitching[pitching.Name == "Total"]
        player_pitching = pitching[pitching.Name != "Total"]
        
        player_pitching = player_pitching.rename(columns = {"Name":"name", "IP":"innings", "SV":"saves", "ER":"earned_runs"})
        player_pitching.innings = player_pitching.innings.apply(lambda x: float(str(x).split(".")[0] +"."+ str(int(str(x).split('.')[-1])/10*3.3).split(".")[-1]))
        player_pitching["game_id"] = game_id
        
        player_pitching["player_id"] = player_pitching.name.apply(lambda x: id_dict[x])
        
        
        
        # Next remove all the pitchers from the batting stats so NL Pitchers don't get batting stats
        pitchers_used = player_pitching.player_id
        player_batting = player_batting[player_batting.player_id.isin(pitchers_used) == False]
        
        
        # Clean the team tables before we append them to the final team stats
        team_batting["game_id"] = game_id
        team_batting["runs_allowed"] = team_batting.apply(lambda x: runs_allowed(x), axis = 1)
        team_batting["win_loss"] = team_batting.apply(lambda x: "W" if x.R > x.runs_allowed else "L", axis = 1)
            # Currently we assume all extra inning games are 10+ innings. If MLB keeps the double header rules then make sure to change to account for 8 inning extra inning games!!!
        team_batting["is_extra_innings"] = 1 if innings > 9 else 0
        team_batting = team_batting.rename(columns = {"Team Name":"team", "R":"runs_scored"})
        
        
        # Append the batting stats to the batting df and the picthing stats to the picthing df
        final_batting_df = final_batting_df.append(player_batting)
        final_pitching_df = final_pitching_df.append(player_pitching)
                # If we ever want to include team pitching stats, below is where we can include team_picthing
        final_team_df = final_team_df.append(team_batting)

        
        # Finally append the date to all dfs
        final_batting_df["Date"] = given_date
        final_pitching_df["Date"] = given_date
        final_team_df['Date'] = given_date
        
        final_team_df = final_team_df[['team'] + [col for col in final_team_df.columns if col not in ["team", "Name"]]]
        

        
    return {"batters":final_batting_df.reset_index(drop=True), "pitchers":final_pitching_df.reset_index(drop=True), "teams":final_team_df.reset_index(drop=True), "date":given_date}
    
    
    

# Transform Daily Stats

### Transform NBA Stats

In [9]:
def transform_nba_stats(nba):
    if len(nba) == 0:
        return pd.DataFrame()
    

    nba_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    nba_data = nba["players"]
    team_data = nba["teams"]
    date = nba["date"]
    
    # Build standard player stats
    nba_stats["Name"] = nba_data["Starters"]
    nba_stats["Team"] = nba_data["team"]
    nba_stats["Points"] = nba_data["PTS"].apply(lambda x: int(x))
    #nba_stats["field_goals_made"] = nba_data["made_field_goals"] 
    #nba_stats["three_pointers"] = nba_data["made_three_point_field_goals"]
    nba_stats["Rebounds"] = nba_data["TRB"].apply(lambda x: int(x))
    nba_stats["Minutes"] = nba_data["MP"].apply(lambda x: float(x))
    nba_stats["Assists"] = nba_data["AST"].apply(lambda x: int(x))
    nba_stats["Turnovers"] = nba_data["TOV"].apply(lambda x: int(x))
    nba_stats["Blocks"] = nba_data["BLK"].apply(lambda x: int(x))
    nba_stats['Steals'] = nba_data["STL"].apply(lambda x: int(x))
    nba_stats["Fouls"] = nba_data["PF"].apply(lambda x: int(x))
    nba_stats["Player ID"] = nba_data["id"]
    
    
    # Build the average player stats that require seperate numerators/denominators, that will calculate final values each day
#     nba_stats["field_goal_percentage_numerator"] = nba_data["made_field_goals"]
#     nba_stats["field_goal_percentage_denominator"] = nba_data["attempted_field_goals"]
    
    nba_stats["Free Throw Percentage Numerator"] = nba_data["FT"].apply(lambda x: int(x))
    nba_stats["Free Throw Percentage Denominator"] = nba_data["FTA"].apply(lambda x: int(x))
    
    
    
    nba_stats["teams_share_of_points_numerator"] = nba_stats["Points"].apply(lambda x: int(x))
    nba_stats["teams_share_of_points_denominator"] = nba_stats["Team"].apply(lambda x: int(team_data[team_data.team == x].PTS.iloc[0]))
    
    
    # Build standard team_stats
    team_stats["Team"] = team_data["team"]
    team_stats["Is Win"] = team_data["is_win"].apply(lambda x: True if x == 1 else False)
    team_stats["Points Scored"] = team_data["PTS"].apply(lambda x: int(x))
    team_stats["Points Allowed"] = team_data.team.apply(lambda x: int(team_data[team_data.opponent == x].PTS.iloc[0]))
    team_stats["Point Differential"] = team_stats["Points Scored"] - team_stats["Points Allowed"]
    team_stats = team_stats.drop(columns = ["Points Scored", "Points Allowed"])
    #team_stats["players_fouled_out"] = team_data["team"].apply(lambda x: len(nba_data[(nba_data.team == x) & (nba_data.personal_fouls >= 6)]))
    
    ### WILL CALCULATE POINTS/SCORE WHEN ASSIGNING TO OWNERSHIP FOR DATES. WILL PUT ALL LISTS OF 8 HIGHEST MINS INTO ONE LIST, THEN CALCULATE THE STDEV
    team_stats["Team Minutes Distribution"] = team_data["team"].apply(lambda x: [nba_stats[nba_stats.Team == x].sort_values(by = "Minutes").iloc[0:8]])
    
    
    # Attatch the date to the dfs for easier reference later on
    nba_stats["date"] = date
    team_stats["date"] = date
    
    # Fillna
    nba_stats = nba_stats.fillna(0)
    team_stats = team_stats.fillna(0)
    
    
    return {"players":nba_stats, "teams":team_stats}

### Transform NHL Stats

In [10]:
def transform_nhl_stats(nhl_data):
    if len(nhl_data) == 0:
        return pd.DataFrame()
    
    goalie_data = nhl_data["Goalies"]
    skater_data = nhl_data["Skaters"][nhl_data["Skaters"].player.isin(goalie_data.player)==False]
    player_data = skater_data.append(goalie_data).reset_index(drop=True)
    team_data = nhl_data["Teams"]
    penalty_data = nhl_data["Penalties"]
    date = nhl_data["date"]
    
    player_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    # Build standard player stats
    player_stats["Name"] = player_data.player.reset_index(drop=True)
    player_stats["Team"] = player_data.team
    player_stats["Goals"] = player_data.goals
    player_stats["Assists"] = player_data.assists
    player_stats["Penalty Minutes"] = player_data.penalty_minutes
    player_stats["Shots"] = player_data.shots
    player_stats["Hits"] = player_data.hits
    player_stats["Blocks"] = player_data.blocks
    player_stats["Bench Minutes"] = player_data.time_on_ice
    player_stats["Fights"] = player_data.player.apply(lambda x: len(penalty_data[(penalty_data.player == x) & (penalty_data.penalty == "Fighting")]))
    player_stats["Goals Against"] = player_data.goals_against 
    player_stats["Shots Against"] = player_data.shots_against
    player_stats["Shutouts"] = player_data.shutouts
    player_stats["Player ID"] = player_data.player_id
    
    # Build the standard team stats
    team_stats["Team"] = team_data.team
    team_stats["Points"] = team_data.points
    team_stats["Goals Scored"] = team_stats.Team.apply(lambda x: player_data[player_data.team == x].team_goals_scored.iloc[0])
    team_stats["Goals Allowed"] = team_stats.Team.apply(lambda x: player_data[player_data.team == x].team_goals_allowed.iloc[0])
    team_stats["Is Win"] = team_stats["Goals Scored"] > team_stats["Goals Allowed"]
    team_stats = team_stats.drop(columns = ["Goals Scored", "Goals Allowed"])
    
    # Build the non-standard team stats
    team_stats["Special Teams Strength Numerator"] = team_data.power_play_goals + team_data.short_handed_goals
    team_stats["Special Teams Strength Denominator"] = team_data.goals
    
    
    # Attatch the date to the dfs for easier reference later on
    player_stats["Date"] = date
    team_stats["Date"] = date
    
    player_stats = player_stats[player_stats.Name != "Empty Net"]
    
    # Fillna
    player_stats = player_stats.fillna(0)
    team_stats = team_stats.fillna(0)
    
    
    return {"players":player_stats, "teams":team_stats}

### Transform NFL Stats

In [97]:
def transform_nfl_stats(nfl_data):
    if len(nfl_data) == 0:
        return pd.DataFrame()
    
    def return_stats(player, stat):
            try:
                ans = return_data[return_data.Player == player][stat].iloc[0]
                return float(ans)
            except:
                return 0
            
    def sitting(player):
        try:
            ans = snaps_data[snaps_data.player == player].sitting_points.iloc[0]
            return ans
        except:
            return 0
        
        
    offensive_data = nfl_data["Offense"]
    return_data = nfl_data["Returns"]
    offensive_data = offensive_data.join(return_data.set_index("player"), on = "player",how = "outer", rsuffix = "off")
    return_teams = offensive_data.teamoff
    offensive_data.team.update(return_teams)
    offensive_data = offensive_data.drop(columns = ["teamoff"])
    
    
    
    defensive_data = nfl_data["Defense"]
    kicking_data = nfl_data["Kicking"]
    snaps_data = nfl_data["Snaps"]
    field_goal_data = nfl_data["Field Goals"]
    penalty_data = nfl_data["Penalties"]
    date = nfl_data["date"]

    
    offensive_stats = pd.DataFrame()
    defensive_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    
    # Convert numbers that need to be calculated on into ints from the strings they are now
    for stat in ["rushing_yards", "rushing_touchdowns", "receiving_yards", "receiving_touchdowns", "total_fumbles", "fumbles_lost", "sacks", "passing_yards", "passing_touchdowns",
                 "interceptions", "fumble_recoveries", "interception_return_yards", "fumble_return_yards"] + ["kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"]:
        try:
            offensive_data[stat] = offensive_data[stat].apply(lambda x: float(x))
            defensive_data[stat] = defensive_data[stat].apply(lambda x: float(x))
        except:
            try:
                offensive_data[stat] = offensive_data[stat].apply(lambda x: float(x))
            except:
                defensive_data[stat] = defensive_data[stat].apply(lambda x: float(x))
                      
    
    # Build standard offensive stats
    offensive_stats["Name"] = offensive_data.player
    offensive_stats["Team"] = offensive_data.team
    offensive_stats["Passing Yards"] = offensive_data.passing_yards
    offensive_stats["Interceptions Thrown"] = offensive_data.interceptions
    offensive_stats["All Purpose Yards"] = offensive_data.rushing_yards + offensive_data.receiving_yards * 0.92
    offensive_stats["All Purpose TDs"] = offensive_data.rushing_touchdowns + offensive_data.receiving_touchdowns * 1.14 + offensive_data.passing_touchdowns * 0.36
    offensive_stats["Fumbles"] = offensive_data.fumbles_lost * 2 + (offensive_data.total_fumbles - offensive_data.fumbles_lost)
    
    snaps_data["percent"] = (snaps_data.offensive_snaps / snaps_data.total_offensive_snaps) * 100
    snaps_data["sitting_points"] = snaps_data.percent.apply(lambda x: 0 if x < 10 else 10 - (x-10))
    
    offensive_stats["Bench Points"] = offensive_stats.Name.apply(lambda x: sitting(x))
    
    
    for return_stat in ["kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"]:
        offensive_stats[return_stat] = offensive_data[return_stat]
        
  
       
        
        
    offensive_stats["Return Yards"] = offensive_stats.kick_return_yards * 0.46 + offensive_stats.punt_return_yards
    offensive_stats["Return TDs"] = offensive_stats.kick_return_touchdowns + offensive_stats.punt_return_touchdowns
    offensive_stats = offensive_stats.drop(columns = ["kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"])
    
    # Build standard defensive stats
    defensive_stats["Team"] = defensive_data.team.unique()
    defensive_stats["Defensive Chaos Index"] = defensive_stats.Team.apply(lambda x: defensive_data[defensive_data.team == x].sacks.sum() * 0.51 + defensive_data[defensive_data.team == x]["interceptions"].sum() + defensive_data[defensive_data.team == x].fumble_recoveries.sum())
    defensive_stats["Turnover Yards"] = defensive_stats.Team.apply(lambda x: defensive_data[defensive_data.team == x].interception_return_yards.sum() + defensive_data[defensive_data.team == x].fumble_return_yards.sum())
    defensive_data["Player ID"] = defensive_data.player_id
    defensive_stats["Tackles"] = defensive_data.total_tackles
    
    
    # Set the index of defensive_stats to the team so we can later join it to team stats on the index
    defensive_stats = defensive_stats.set_index("Team")
    
    # Build standard kicking stats
    kicking_data.punt_yards = kicking_data.punt_yards.apply(lambda x: float(x))
    field_goal_data.distance = field_goal_data.distance.apply(lambda x: float(x))
    punting_stats = pd.DataFrame()
    placekicking_stats = pd.DataFrame()
    
    punting_stats["Name"] = kicking_data[kicking_data.punt_yards >0].player.unique()
    punting_stats["Player ID"] = punting_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].player_id.iloc[0])
    punting_stats["Team"] = punting_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].team.iloc[0])
    placekicking_stats["Name"] = field_goal_data.kicker.unique()
    placekicking_stats["Player ID"] = placekicking_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].player_id.iloc[0])
    placekicking_stats["Team"] = placekicking_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].team.iloc[0])
    
    punting_stats["Punt Yards"] = punting_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].punt_yards.sum())
    placekicking_stats["Field Goal Yards"] = placekicking_stats.Name.apply(lambda x: field_goal_data[field_goal_data.kicker == x].distance.sum())
    total_kicking_stats = pd.concat([punting_stats, placekicking_stats])
    
    
    # Build Standard Team Stats
    team_stats["Team"] = offensive_data.team.unique()
    
    team_stats["Point Differential"] = team_stats.Team.apply(lambda x: offensive_data[offensive_data.team == x].points_scored.iloc[0] - offensive_data[offensive_data.team == x].points_allowed.iloc[0] if pd.isna(x) == False else 0)
    team_stats["win"] = team_stats["Point Differential"].apply(lambda x: 1 if x > 0 else 0)
    team_stats["Penalty Yards"] = team_stats.Team.apply(lambda x: penalty_data[penalty_data.team == x].penalty_yards.iloc[0] if pd.isna(x) ==False else 0)
    
    # Set the index of team stats to the team so we can join it to the defensive stats on the index
    team_stats = team_stats.set_index("Team")    
    
    individual_stats = offensive_stats.append(total_kicking_stats)
    team_based_stats = defensive_stats.join(team_stats)
    
    # Now that the team stats have been joined on the index we can reset the index
    team_based_stats = team_based_stats.reset_index()
    
    # Attatch the date to each df for easier reference later on
    individual_stats["Date"] = date
    team_based_stats["Date"] = date
    
    # Fillna
    individual_stats = individual_stats.fillna(0)
    team_based_stats = team_based_stats.fillna(0)
    
    # We are currently unable to assign kickers to a specific team
    individual_stats.team = individual_stats.Team.apply(lambda x: "Unknown" if x == 0 else x)

    individual_stats["Player ID"] = individual_stats.Name + individual_stats.Team
    
    return {"players":individual_stats.reset_index(drop = True), "teams":team_based_stats.reset_index(drop = True)}
    

### Transform MLB Stats

In [83]:
def transform_mlb_stats(mlb_data):
    batting_data = mlb_data["batters"]
    pitching_data = mlb_data["pitchers"]
    team_data= mlb_data["teams"]
    date = mlb_data["date"]
    
    batting_stats = pd.DataFrame()
    pitching_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    # Build standard batting_stats
    batting_stats["Name"] = batting_data.name
    batting_stats["Plate Appearences"] = batting_data.plate_appearences
    batting_stats["Hits"] = batting_data.hits
    batting_stats["Walks"] = batting_data.walks
    batting_stats["Runs"] = batting_data.runs
    batting_stats["RBIs"] = batting_data.runs_batted_in
    batting_stats["XBH"] = batting_data.doubles + batting_data.triples + batting_data.home_runs
    batting_stats["Home Runs"] = batting_data.home_runs
    batting_stats["Stolen Bases"] = batting_data.stolen_bases
    batting_stats["Sacrifices"] = batting_data.sacrifices
    batting_stats["Strikeouts"] = batting_data.strikeouts
    batting_stats["Player ID"] = batting_data.player_id
    batting_stats["Total Bases"] = batting_data.total_bases
    batting_stats["Sitting"] = 0 # FIX THIS BEFORE THE SEASON BEGINS
   
    
    # Build the standard pitching stats
    pitching_stats["Name"] = pitching_data.name
    pitching_stats["Innings"] = pitching_data.innings
    pitching_stats["Saves"] = pitching_data.saves
    pitching_stats["Player ID"] = pitching_data.player_id
    pitching_stats["Strikeouts"] = pitching_data.SO
    
    # Build the non-standard pitching stats
    pitching_stats["ERA Numerator"] = pitching_data.earned_runs
    pitching_stats["ERA Denominator"] = pitching_data.innings
    

    
    # Build standard team stats
    team_stats["Team"] = team_data.team
    team_stats["Is Win"] = team_data.win_loss.apply(lambda x: True if x == "W" else False)
    team_stats["Run Differential"] = team_data.runs_scored - team_data.runs_allowed
    team_stats["Is Extra Inning Game"] = team_data.is_extra_innings.apply(lambda x: True if x == 1 else False)
    
    
    # Combine the batting and pitching stats into one df for all players
    player_stats = pd.concat([batting_stats, pitching_stats])
  
    
    
    return {"players":player_stats, "teams":team_stats}
    

# Scoring/Running Functions

### Get All Stats For a Given Day

In [86]:
x = get_daily_league_stats(27,10,2022)

In [90]:
x["NFL"]["teams"]

,Team,Defensive Chaos Index,Turnover Yards,Tackles,Point Differential,win,Penalty Yards,Date
0,BAL,2.53,0.0,2,5.0,1,101.0,2022-10-27
1,TAM,1.53,0.0,3,-5.0,0,35.0,2022-10-27


In [13]:
def get_daily_league_stats(day, month, year):
    raw_nfl_stats = get_daily_nfl_stats(day, month, year)
    nfl_stats = transform_nfl_stats(raw_nfl_stats)
    
    raw_nhl_stats = get_daily_nhl_stats(day, month, year)
    nhl_stats = transform_nhl_stats(raw_nhl_stats)
    
    raw_nba_stats = get_daily_nba_stats(day, month, year)
    nba_stats = transform_nba_stats(raw_nba_stats)
    
    # If there are no games on the given day, unlike the other sports the MLB run will not fail quietly --> set up a try except with a visual warning
    try:
        raw_mlb_stats = get_daily_mlb_stats(day, month, year)
        mlb_stats = transform_mlb_stats(raw_mlb_stats)
        return {"NFL":nfl_stats, "NHL":nhl_stats, "NBA":nba_stats, "MLB":mlb_stats}
        #clear_output(wait = True)
    except:
        print("An error was thrown pulling MLB stats. If the MLB is in season, investigate the error further.")
        time.sleep(4)
        mlb_stats = pd.DataFrame()
        clear_output(wait = False)

        
    return {"NFL":nfl_stats, "NHL":nhl_stats, "NBA":nba_stats, "MLB":mlb_stats}

### Assigning Stats to Teams

In [14]:
league_transactions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/League Transactions.xlsx", header = 5, sheet_name = "Transactions").iloc[:,3:12]

In [15]:
def get_team_from_transactions(participant, date, transactions):
    output = pd.DataFrame()
    
    relevant_transactions = transactions[(transactions.Date <= date) & (transactions.Participant == participant)]
    
    # A player is actively on a team on a certain date if on/before that date there are more transactions for that player within that team adding them than there is dropping them
    relevant_transactions["on_team"] = relevant_transactions.Player.apply(lambda x: 1 if len(relevant_transactions[(relevant_transactions.Player == x) & (relevant_transactions["Transaction Type"] == "Add")]) >  len(relevant_transactions[(relevant_transactions.Player == x) & (relevant_transactions["Transaction Type"] == "Drop")]) else 0)
    output = output.append(relevant_transactions)
    
    
    return output[output.on_team == 1][["Participant", "Player", "Date", "League", "Position", "Player ID"]]

In [16]:
def assign_daily_team_stats(participant, daily_data, date):
    team = get_team_from_transactions(participant, date, league_transactions)
    output = {}
    
    
    for league in daily_data.keys():
        # Filter all daily data to a specific league, and then grab the players for a specific participant for just that league
        league_data = daily_data[league]
        league_team = team[team.League == league]
  
        if len(league_data) > 0:
            player_stats = league_data["players"][league_data["players"]["Player ID"].isin(league_team["Player ID"])]
            player_stats["Position"] = player_stats["Player ID"].apply(lambda x: league_team[league_team["Player ID"] == x].Position.iloc[0])
            
            
            team_stats = league_data["teams"][league_data["teams"].Team.isin(league_team.Player)]
            team_stats["Position"] = team_stats.Team.apply(lambda x: league_team[league_team.Player == x].Position.iloc[0])
            
        else:
            player_stats = pd.DataFrame()
            team_stats = pd.DataFrame()
            
        try:
            player_stats = player_stats[[col for col in player_stats.columns if col != "Player ID"] + ["Player ID"]]
            team_stats = team_stats[[col for col in team_stats.columns if col != "Player ID"] + ["Player ID"]] 
        except:
            None
        output[league] = {"players":player_stats, "teams":team_stats}
        
    
    return output

If we ever need to create an empty stats dictionary to begin with "yesterdays stats" run the below cell with the day and month updated

In [17]:
# names = ["Noah", "Julia", "Jared", "Alex", "Josh", "Jordan", "Dalton", "Jesse", "Raphy"]
# day = 31
# month = 12

# annual_stats = {"NFL":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "NHL":{"players":pd.DataFrame(), "teams":pd.DataFrame()},
#                                          "NBA":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "MLB":{"players":pd.DataFrame(), "teams":pd.DataFrame()}}

# league_stats_by_participant = {}
# for name in names:
#     league_stats_by_participant[name] = {"NFL":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "NHL":{"players":pd.DataFrame(), "teams":pd.DataFrame()},
#                                          "NBA":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "MLB":{"players":pd.DataFrame(), "teams":pd.DataFrame()}}

# pkl.dump(league_stats_by_participant.copy(), open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/League Stats By Participant/League Stats By Participant {}-{}.pkl".format(str(month).zfill(2), str(day).zfill(2)), "wb"))
# pkl.dump(annual_stats.copy(), open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Annual League Stats/Annual Stats {}-{}.pkl".format(str(month).zfill(2), str(day).zfill(2)), "wb"))

In [18]:
def update_annual_team_stats(day, month, year):
    indent_counter = 0
    date = "{}/{}/{}".format(str(month).zfill(2), str(day).zfill(2), str(year).zfill(2))
    
    # Pull todays stats - they will eventually be appended to all the stats we've collected up until the day before
    day_stats = get_daily_league_stats(day, month, year)

    # Pull the pickle of stats ending yesterday as our base to append to
    yesterdays_date = (datetime.strptime(date, "%m/%d/%Y") - timedelta(1)).strftime("%m-%d")
    yesterdays_data = pkl.load(open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/League Stats By Participant/League Stats By Participant {}.pkl".format(yesterdays_date), "rb"))

    todays_data = {}
    # For each participant in the league, slice the stats of the team they own on the given day, then append the stats to the respective df of stats for the year
    # of the participant, league, and player type 
    for participant in league_transactions.Participant.unique():
        todays_data[participant] = {}
        team_stats = assign_daily_team_stats(participant, day_stats, date)
        for league in team_stats.keys():
            todays_data[participant][league] = {}
            for player_type in team_stats[league].keys():
                df = team_stats[league][player_type]
                todays_data[participant][league][player_type] = df

              
    for participant in todays_data:
        for league in todays_data[participant]:
            for player_type in todays_data[participant][league]:
                yesterdays_data[participant][league][player_type] = yesterdays_data[participant][league][player_type].append(todays_data[participant][league][player_type])
                
    

    # Finally, pickle the dictionary of all of the stats for the year, to be used as yesterdays stats tomorrow
    pkl_date = "{}-{}".format(str(month).zfill(2), str(day).zfill(2))
    pkl.dump(yesterdays_data, open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/League Stats By Participant/League Stats By Participant {}.pkl".format(pkl_date), "wb")) 
    
    
    # Next, write the stats to an excel page that will eventually be sent out to the league
    writer = pd.ExcelWriter('/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Scoresheet Backups/League File {}.xlsx'.format(str(month) + str(day) + str(2021)), engine='xlsxwriter') 
    yesterdays_annual_stats = pkl.load(open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Annual League Stats/Annual Stats {}.pkl".format(yesterdays_date), "rb"))
    todays_annual_stats = yesterdays_annual_stats.copy()

    for league in todays_annual_stats.keys():
        try:
            league_stats = day_stats[league]
            if len(league_stats) < 1:
                league_stats = {"players":pd.DataFrame(), "teams":pd.DataFrame()}
        except:
            league_stats = {"players":pd.DataFrame(), "teams":pd.DataFrame()}
            
        
        todays_annual_stats[league]["players"] = todays_annual_stats[league]["players"].append(league_stats["players"])
        todays_annual_stats[league]["teams"] = todays_annual_stats[league]["teams"].append(league_stats["teams"])


        todays_annual_stats[league]["players"].to_excel(writer, sheet_name='Sheet1', startcol=indent_counter, startrow=1, index = False) # Default position, cell A1.
        indent_counter += len(todays_annual_stats[league]["players"].columns) + 3
        todays_annual_stats[league]["teams"].to_excel(writer, sheet_name='Sheet1', startcol= indent_counter, startrow=1, index = False)
        indent_counter += len(todays_annual_stats[league]["teams"].columns) + 3
    writer.save() 
    
    pkl.dump(todays_annual_stats, open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Annual League Stats/Annual Stats {}.pkl".format(pkl_date), "wb"))
    
    # Even though its called yesterdays data, its actually yesterdays data appended with todays data, making it current and correct
    return yesterdays_data, todays_annual_stats

In [19]:
def aggregate_stats_by_participant(league_data_by_participant, date):
    participants = league_data_by_participant.keys()
    output = {k:pd.DataFrame() for k in ["NFL", "NHL", "NBA", "MLB"]}
    
    # Aggregate NFL Stats
    for participant in participants:
        
        # Seperate the bench players and the starters so that we can only give the bench players their bench points, not regular points as well.
        if len(league_data_by_participant[participant]["NFL"]["players"]) > 0:
            starters_data = league_data_by_participant[participant]["NFL"]["players"][league_data_by_participant[participant]["NFL"]["players"].Position != "Bench"]
            sitters_data = league_data_by_participant[participant]["NFL"]["players"][league_data_by_participant[participant]["NFL"]["players"].Position == "Bench"]

            grouped_starters_data = starters_data.groupby(by = "Player ID").sum()
            final_starters_data = grouped_starters_data.sum().drop(columns = ["Bench Points"])

            grouped_sitters_data = sitters_data.groupby(by = "Player ID").sum()
            final_sitters_data = grouped_sitters_data.sum()["Bench Points"]

            # Attatch the bench points to the full aggregated NFL Stats and rename it
            final_starters_data["Bench Points"] = final_sitters_data
            final_nfl_players_data = final_starters_data
            final_nfl_players_data["Participant"] = participant

            # Attatch the individual defense and its stats to the aggregated NFL stats
            current_defense = league_transactions[(league_transactions.Participant == participant) & (league_transactions.League == "NFL") & (league_transactions.Position == "D")].Player.iloc[-1]
            defensive_data = league_data_by_participant[participant]["NFL"]["teams"][league_data_by_participant[participant]["NFL"]["teams"].Position == "D"][["Team", "Defensive Chaos Index", "Turnover Yards"]]

            grouped_defensive_data = defensive_data.groupby(by = "Team").sum()
            final_defensive_data = grouped_defensive_data.sum()

            final_nfl_players_data = final_nfl_players_data.append(final_defensive_data)
        
                
            # Aggregate NFL Team Stats
            current_team = league_transactions[(league_transactions.Participant == participant) & (league_transactions.League == "NFL") & (league_transactions.Position == "Team")].Player.iloc[-1]
            team_data = league_data_by_participant[participant]["NFL"]["teams"][league_data_by_participant[participant]["NFL"]["teams"].Position == "Team"][["Team", "Point Differential", "win", "Penalty Yards"]].rename(columns = {"win":"Wins"})
            grouped_team_data = team_data.groupby(by="Team").sum()
            final_team_data = grouped_team_data.sum()

            # Attatch the aggregated team data to the full aggregated NFL stats
            final_nfl_players_data = final_nfl_players_data.append(final_team_data)
            final_nfl_players_data = final_nfl_players_data[["Participant"] + [col for col in final_nfl_players_data.index if col != "Participant"]]

            output["NFL"] = output["NFL"].append(final_nfl_players_data, ignore_index = True)
        else:
            output["NFL"] = output["NFL"].append(pd.DataFrame())
            

        
    # Aggregate NHL Stats
    for participant in participants:
        
        if len(league_data_by_participant[participant]["NHL"]["players"]) > 0:
            # Seperate the bench players and the starters so that we can only give the bench players their bench points, not regular points as well.
            starters_data = league_data_by_participant[participant]["NHL"]["players"][league_data_by_participant[participant]["NHL"]["players"].Position != "Bench"]
            sitters_data = league_data_by_participant[participant]["NHL"]["players"][league_data_by_participant[participant]["NHL"]["players"].Position == "Bench"]

            grouped_starters_data = starters_data.groupby(by = "Player ID").sum()
            final_starters_data = grouped_starters_data.sum().drop(index = ["Bench Minutes"])

            sitters_data["Bench Points"] = sitters_data["Bench Minutes"].apply(lambda x: 0 if x<6 else 5-(x-6))
            sitters_data = sitters_data.drop(columns = ["Bench Minutes"])

            grouped_sitters_data = sitters_data.groupby(by = "Player ID").sum()
            final_sitters_data = grouped_sitters_data.sum()["Bench Points"]

            # Attatch the bench points to the full aggregated NHL Stats and rename it
            final_starters_data["Bench Points"] = final_sitters_data
            final_nhl_players_data = final_starters_data
            final_nhl_players_data["Participant"] = participant

            # Aggregate NHL Team Stats
            current_team = league_transactions[(league_transactions.Participant == participant) & (league_transactions.League == "NHL") & (league_transactions.Position == "Team")].Player.iloc[-1]
            team_data = league_data_by_participant[participant]["NHL"]["teams"][league_data_by_participant[participant]["NHL"]["teams"].Position == "Team"]
            grouped_team_data = team_data.groupby(by="Team").sum()
            final_team_data = grouped_team_data.sum()
            final_team_data["Special Teams Strength"] = final_team_data["Special Teams Strength Numerator"]/final_team_data["Special Teams Strength Denominator"]
            final_team_data = final_team_data.drop(index = ["Special Teams Strength Numerator", "Special Teams Strength Denominator"])

            # Attatch the aggregated team data to the full aggregated NHL stats
            final_nhl_players_data = final_nhl_players_data.append(final_team_data)
            final_nhl_players_data = final_nhl_players_data[["Participant"] + [col for col in final_nhl_players_data.index if col != "Participant"]]

            output["NHL"] = output["NHL"].append(final_nhl_players_data, ignore_index = True) 


        # Make the NHL Stats look nice
        for col in output["NHL"].columns:
                if col not in ["Participant", "Bench Points", "Special Teams Strength"]:
                    output["NHL"][col] = output["NHL"][col].apply(lambda x: int(x))

        output["NHL"]["Bench Points"] = output["NHL"]["Bench Points"].apply(lambda x: round(x, 2))
        output["NHL"]["Special Teams Strength"] = output["NHL"]["Special Teams Strength"].apply(lambda x: round(x*100, 2))
        
    else:
        output["NHL"] = output["NHL"].append(pd.DataFrame())


    # Aggregate NBA Stats
    for participant in participants:
        
        if len(league_data_by_participant[participant]["NBA"]["players"]) > 0:
            # Seperate the bench players and the starters so that we can only give the bench players their bench points, not regular points as well.
            starters_data = league_data_by_participant[participant]["NBA"]["players"][league_data_by_participant[participant]["NBA"]["players"].Position != "Bench"]
            sitters_data = league_data_by_participant[participant]["NBA"]["players"][league_data_by_participant[participant]["NBA"]["players"].Position == "Bench"]

            grouped_starters_data = starters_data.groupby(by = "Player ID").sum()
            final_starters_data = grouped_starters_data.sum().drop(index = ["Bench Minutes"])
            final_starters_data["Free Throw %"] = final_starters_data["Free Throw Percentage Numerator"]/final_starters_data["Free Throw Percentage Denominator"]
            final_starters_data["Free Throw %"] = str(round(final_starters_data["Free Throw %"]*100,2)) + "%"
            final_starters_data = final_starters_data.drop(index = ["Free Throw Percentage Numerator", "Free Throw Percentage Denominator"])

            # Build sitting points and Make the values look nicer
            sitters_data["Bench Points"] = sitters_data["Bench Minutes"].apply(lambda x: 0 if x<5 else 5-(x-5))
            sitters_data = sitters_data.drop(columns = ["Bench Minutes"])

            grouped_sitters_data = sitters_data.groupby(by = "Player ID").sum()
            final_sitters_data = grouped_sitters_data.sum()["Bench Points"]

            # Attatch the bench points to the full aggregated NBA Stats and rename it
            final_starters_data["Bench Points"] = final_sitters_data
            final_nba_players_data = final_starters_data
            final_nba_players_data["Participant"] = participant


            # Aggregate NBA Team Stats
            current_team = league_transactions[(league_transactions.Participant == participant) & (league_transactions.League == "NBA") & (league_transactions.Position == "Team")].Player.iloc[-1]
            team_data = league_data_by_participant[participant]["NBA"]["teams"][league_data_by_participant[participant]["NBA"]["teams"].Position == "Team"]

            grouped_team_data = team_data.groupby(by="Team").sum()
            final_team_data = grouped_team_data.sum()
            final_team_data["Team Minutes Distribution"] = round(statistics.stdev([i for sublist in [list(l[0]) for l in team_data["Team Minutes Distribution"]] for i in sublist]),2)


            # Attatch the aggregated team data to the full aggregated NFL stats
            final_nba_players_data = final_nba_players_data.append(final_team_data)
            final_nba_players_data = final_nba_players_data[["Participant"] + [col for col in final_nba_players_data.index if col != "Participant"]]


            output["NBA"] = output["NBA"].append(final_nba_players_data, ignore_index = True) 
            
        else:
            output["NBA"] = output["NBA"].append(pd.DataFrame())
    
    # Make the NBA Stats look nice
    for col in output["NBA"].columns:
            if col not in ["Participant", "Free Throw %", "Bench Points", "Team Minutes Distribution"]:
                output["NBA"][col] = output["NBA"][col].apply(lambda x: int(x))
    
    output["NBA"] = output["NBA"].rename(columns = {"Is Win":"Wins"}) 
    
    
    # Aggregate MLB Stats
    for participant in participants:
        
        if len(league_data_by_participant[participant]["MLB"]["players"]) > 0:
            # Seperate the bench players and the starters so that we can only give the bench players their bench points, not regular points as well.
            starters_data = league_data_by_participant[participant]["MLB"]["players"][league_data_by_participant[participant]["MLB"]["players"].Position != "Bench"]
            sitters_data = league_data_by_participant[participant]["MLB"]["players"][league_data_by_participant[participant]["MLB"]["players"].Position == "Bench"]

            grouped_starters_data = starters_data.groupby(by = "Player ID").sum()
            final_starters_data = grouped_starters_data.sum().drop(index = ["Sitting"])
            final_starters_data["ERA"] = round(final_starters_data["ERA Numerator"]/final_starters_data["ERA Denominator"] * 9, 2)
            final_starters_data = final_starters_data.drop(index = ["ERA Numerator", "ERA Denominator"])


            sitters_data["Sitting"] = 0 #FIX THIS BEFORE THE MLB SEASON BEGINS

            grouped_sitters_data = sitters_data.groupby(by = "Player ID").sum()
            final_sitters_data = grouped_sitters_data.sum()["Sitting"]

            # Attatch the bench points to the full aggregated MLB Stats and rename it
            final_starters_data["Bench Points"] = final_sitters_data
            final_mlb_players_data = final_starters_data
            final_mlb_players_data["Participant"] = participant


            # Aggregate MLB Team Stats
            current_team = league_transactions[(league_transactions.Participant == participant) & (league_transactions.League == "MLB") & (league_transactions.Position == "Team")].Player.iloc[-1]
            team_data = league_data_by_participant[participant]["MLB"]["teams"][league_data_by_participant[participant]["MLB"]["teams"].Position == "Team"]

            grouped_team_data = team_data.groupby(by="Team").sum()
            final_team_data = grouped_team_data.sum()

            # Attatch the aggregated team data to the full aggregated NFL stats
            final_mlb_players_data = final_mlb_players_data.append(final_team_data)
            final_mlb_players_data = final_mlb_players_data[["Participant"] + [col for col in final_mlb_players_data.index if col != "Participant"]] 

            output["MLB"] = output["MLB"].append(final_mlb_players_data, ignore_index = True)


        for col in output["MLB"].columns:
            if col not in ["Participant", "Innings", "ERA"]:
                output["MLB"][col] = output["MLB"][col].apply(lambda x: int(x))
        
        else:
            output["MLB"] = output["MLB"].append(pd.DataFrame())
            
    day = date.split("/")[0].zfill(2)
    month = date.split("/")[1].zfill(2)

    pkl.dump(output, open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Aggregated Stats/Aggregated Stats {}".format(day + "-" + month), "wb"))
    
    return output
    

In [20]:
def score_league(aggregated_stats, date):
    
    # Score NFL Data
    nfl_data = aggregated_stats["NFL"]
    if len(nfl_data) > 0:
        reverse_categories = ["Interceptions Thrown", "Fumbles", "Penalty Yards"]
        nfl_points = pd.DataFrame()
        nfl_points["Participant"] = nfl_data.Participant


        for col in [k for k in nfl_data.columns if k != "Participant"]:
            if col not in reverse_categories:
                nfl_points[col] =  nfl_data[col].rank(na_option = "bottom")-1
            else:
                nfl_points[col] = nfl_data[col].rank(na_option = "bottom", ascending = False)-1
    else:
        nfl_points = pd.DataFrame()
            
    # Score NHL Data
    nhl_data = aggregated_stats["NHL"]
    if len(nhl_data) > 0:
        reverse_categories = ["Penalty Minutes", "Goals Against"]
        nhl_points = pd.DataFrame()
        nhl_points["Participant"] = nhl_data.Participant

        for col in [k for k in nhl_data.columns if k != "Participant"]:
            if col not in reverse_categories:
                nhl_points[col] = nhl_data[col].rank(na_option = "bottom")-1
            else:
                nhl_points[col] = nhl_data[col].rank(na_option = "bottom", ascending = False)-1
    else:
        nhl_points = pd.DataFrame()
            
    # Score NBA Data
    nba_data = aggregated_stats["NBA"]
    if len(nba_data) > 0:
        reverse_categories = ["Turnovers", "Team Minutes Distribution"]
        nba_points = pd.DataFrame()
        nba_points["Participant"] = nba_data.Participant

        for col in [k for k in nba_data.columns if k != "Participant"]:
            if col not in reverse_categories:
                nba_points[col] = nba_data[col].rank(na_option = "bottom")-1
            else:
                nba_points[col] = nba_data[col].rank(na_option = "bottom", ascending = False)-1
    else:
        nba_points = pd.DataFrame()
            
    # Score MLB Data
    mlb_data = aggregated_stats["MLB"]
    if len(mlb_data) > 0:
        reverse_categories = ["Strikeouts", "ERA"]
        mlb_points = pd.DataFrame()
        mlb_points["Participant"] = mlb_data.Participant

        for col in [k for k in mlb_data.columns if k != "Participant"]:
            if col not in reverse_categories:
                mlb_points[col] = mlb_data[col].rank(na_option = "bottom")-1
            else:
                mlb_points[col] = mlb_data[col].rank(na_option = "bottom", ascending = False)-1
    else:
        mlb_points = pd.DataFrame()
            
    
    output = {"NFL":nfl_points, "NHL":nhl_points, "NBA":nba_points, "MLB":mlb_points}
    
            
    return output

In [35]:
def update_league(date):
    day = date.split("/")[1].zfill(2)
    month = date.split("/")[0].zfill(2)
    year = date.split("/")[2]
    
    annual_stats = update_annual_team_stats(int(day), int(month), int(year))
    stats_by_participant = annual_stats[0]
    
    aggregated_stats = aggregate_stats_by_participant(stats_by_participant, date)
    return aggregated_stats
    
    current_scores = score_league(aggregated_stats, date)
    
    
    # Write everything to excel
    
    # To be used in writing files to excel   
    day = date.split("/")[1].zfill(2)
    month = date.split("/")[0].zfill(2)
    year = date.split("/")[2]
            
    # Write aggregated stats to excel
    writer = pd.ExcelWriter('/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Scoresheet Backups/League File {}.xlsx'.format(str(month) + str(day) + str(2021)), engine='xlsxwriter')
    indent_counter_one = 3
    indent_counter_two = 3
    indent_counter_three = 3
    
    
    for league in aggregated_stats.keys():
        
        annual_stats[1][league]["players"].to_excel(writer, sheet_name='Raw Stats', startcol=indent_counter_three, startrow=1, index = False) # Default position, cell A1.
        indent_counter_three += len(annual_stats[1][league]["players"].columns) + 3
        annual_stats[1][league]["teams"].to_excel(writer, sheet_name='Raw Stats', startcol= indent_counter_three, startrow=1, index = False)
        indent_counter_three += len(annual_stats[1][league]["teams"].columns) + 3
        
        aggregated_stats[league].to_excel(writer, sheet_name='Aggregated Season Stats', startcol=1, startrow=indent_counter_one, index = False) # Default position, cell A1.
        indent_counter_one += len(aggregated_stats[league]) + 4
        
        current_scores[league].to_excel(writer, sheet_name='Total Points', startcol=1, startrow=indent_counter_two, index = False) # Default position, cell A1.
        indent_counter_two += len(current_scores[league]) + 4
        
    writer.save()   
    


In [22]:
for num in range(8,10):
    print(num)
    update_league("10/{}/2022".format(str(num)))

8
9
